# Avocado Data Analysis Notebook
ALT-TAB LABS LLP &copy; 2020 All Rights Reserved

In [3]:
# show files in current directory
import os
os.listdir()

['.ipynb_checkpoints',
 'avocado-format.png',
 'avocado-submission.csv',
 'avocado-test.csv',
 'avocado-train.csv',
 'avocado_deep model.ipynb',
 'avocado_notebook.ipynb',
 'avocado_notebook_other_models.ipynb',
 'Given',
 'Models',
 'pyctfsglib.py',
 '__pycache__']

In [45]:
# load csvs
import pandas as pd
df = pd.read_csv('avocado-train.csv', index_col='id')

df.head()

,DataBatch,Date,TotalVolume,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags,type,region,AveragePrice
id,,,,,,,,,,,,,
0,7,2016-11-06,183542.31,98949.98,22891.61,95.00,61605.72,43571.99,17499.01,534.72,conventional,NewOrleansMobile,1.49
1,24,2017-07-16,224434.92,42951.31,120360.02,131.85,60991.74,53141.81,3621.04,4228.89,conventional,HarrisburgScranton,1.38
2,51,2015-01-04,3846.69,1500.15,938.35,0.00,1408.19,1071.35,336.84,0.00,organic,Atlanta,1.76
3,22,2015-07-26,91825.07,1679.28,45615.48,741.77,43788.54,43788.54,0.00,0.00,conventional,BuffaloRochester,1.39
4,50,2015-01-11,54644.32,1491.88,33759.12,1325.17,18068.15,12165.94,5902.21,0.00,conventional,Pittsburgh,1.54


In [46]:
# Convert strings to numbers
c = ['TotalVolume',"4046","4225","4770","TotalBags","SmallBags","LargeBags","XLargeBags","AveragePrice"]
for column in c:
    df[column] = pd.to_numeric(df[column])

# Select data for learning
features = c[:-1]+["type","region"]
X = df[features]
Y = df["AveragePrice"]

# Split training into some for training and some for testing
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
print("training split: ", len(X_train), "; test split: ", len(X_test))

training split:  8558 ; test split:  4216


In [47]:
### Dealing with Missing Values ########################################################
# https://www.kaggle.com/alexisbcook/missing-values
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

def dropColumns(X_train, X_valid):
    # Get names of columns with missing values
    cols_with_missing = [col for col in X_train.columns
                        if X_train[col].isnull().any()]

    # Drop columns in training and validation data
    reduced_X_train = X_train.drop(cols_with_missing, axis=1)
    reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)
    return reduced_X_train, reduced_X_valid


def imputing(X_train, X_valid):
    # Imputation
    from sklearn.impute import SimpleImputer
    my_imputer = SimpleImputer()
    imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
    imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

    # Imputation removed column names; put them back
    imputed_X_train.columns = X_train.columns
    imputed_X_valid.columns = X_valid.columns
    return imputed_X_train, imputed_X_valid

def imputePlus(X_train, X_valid):
    X_train_plus = X_train.copy()
    X_valid_plus = X_valid.copy()

    # Make new columns indicating what will be imputed
    # Get names of columns with missing values
    cols_with_missing = [col for col in X_train.columns
                        if X_train[col].isnull().any()]
    for col in cols_with_missing:
        X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
        X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

    # Imputation
    from sklearn.impute import SimpleImputer
    my_imputer = SimpleImputer()
    imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
    imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

    # Imputation removed column names; put them back
    imputed_X_train_plus.columns = X_train_plus.columns
    imputed_X_valid_plus.columns = X_valid_plus.columns
    return imputed_X_train_plus, imputed_X_valid_plus

### Dealing with Categorial Variables ##################################################
def getCategories(X_train):
    # Get list of categorical variables
    s = (X_train.dtypes == 'object')
    object_cols = list(s[s].index)
    return object_cols

def dropCategories(X_train, X_valid):
    object_cols = getCategories(X_train)
    drop_X_train = X_train.select_dtypes(exclude=['object'])
    drop_X_valid = X_valid.select_dtypes(exclude=['object'])
    return drop_X_train, drop_X_valid, Y_train, Y_valid

# Label Encoding
def labelEncoding(X_train, X_valid):
    object_cols = getCategories(X_train)

    from sklearn.preprocessing import LabelEncoder
    # Make copy to avoid changing original data 
    label_X_train = X_train.copy()
    label_X_valid = X_valid.copy()

    # Apply label encoder to each column with categorical data
    label_encoder = LabelEncoder()
    for col in object_cols:
        label_X_train[col] = label_encoder.fit_transform(X_train[col])
        label_X_valid[col] = label_encoder.transform(X_valid[col])

    return label_X_train, label_X_valid, Y_train, Y_valid

### One Hot Encoding
def oneHotEncoding(X_train, X_valid, debug=True):
    object_cols = getCategories(X_train)

    from sklearn.preprocessing import OneHotEncoder
    # Apply one-hot encoder to each column with categorical data
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
    OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

    # One-hot encoding removed index; put it back
    OH_cols_train.index = X_train.index
    OH_cols_valid.index = X_valid.index

    # Remove categorical columns (will replace with one-hot encoding)
    num_X_train = X_train.drop(object_cols, axis=1)
    num_X_valid = X_valid.drop(object_cols, axis=1)

    # Add one-hot encoded columns to numerical features
    OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
    OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

    return OH_X_train, OH_X_valid

In [48]:
# View the data!
#df.head()
X_train, X_test = oneHotEncoding(X_train, X_test)
X_train, X_test = imputePlus(X_train, X_test)

In [49]:
X_train.head()
#y.head()

,TotalVolume,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags,0,1,...,46,47,48,49,50,51,52,53,54,55
0,174079.23,1225.77,102134.85,3082.60,67636.01,60761.60,6851.61,22.80,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,24727.46,201.46,24405.82,0.00,120.18,30.00,90.18,0.00,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5570915.26,2780859.66,2108450.36,121614.31,559990.93,520299.26,36501.18,3190.49,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,634.09,228.39,24.04,0.00,381.66,381.66,0.00,0.00,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2591981.76,1014382.60,1205553.71,74862.42,297183.03,280841.88,13203.35,3137.80,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
# Preprocessing of training data, fit model 
from sklearn.model_selection import train_test_split
X_train_full, X_test_full, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
print("training split: ", len(X_train), "; test split: ", len(X_test))
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]
# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]
# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()
##############################################################################################################
# Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=0)
from sklearn.metrics import mean_absolute_error
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

my_pipeline.fit(X_train, Y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_test)

# Evaluate the model
score = mean_absolute_error(Y_test, preds)
print('MAE:', score)

training split:  8558 ; test split:  4216
MAE: 0.14552938804554078


# Regression Time!

In [35]:
# Sci Kit Learn! Scientific Computing library for python
from sklearn import *

In [50]:
# Pick the regression model we want to use

import sklearn.tree as tree
models = [
    tree.DecisionTreeRegressor(random_state=2020),
    ensemble.RandomForestRegressor(random_state=2020, n_estimators=100)
]

currScore = 0
model = None
for i in models:
    print(i)
    i.fit(X_train,Y_train)
    score = i.score(X_test,Y_test)
    if score > currScore:
        model = i
        currScore = score

# https://www.kdnuggets.com/2016/10/beginners-guide-neural-networks-python-scikit-learn.html/2
'''
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(Xtrain)
Xtrain = scaler.transform(Xtrain)
Xtest = scaler.transform(Xtest)

from sklearn.neural_network import MLPClassifier
scaler = StandardScaler()
model = MLPClassifier(hidden_layer_sizes=(len(features),len(features),len(features)))
'''


DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=2020, splitter='best')
RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=2020, verbose=0, warm_start=False)


'\nfrom sklearn.preprocessing import StandardScaler\nscaler = StandardScaler()\n# Fit only to the training data\nscaler.fit(Xtrain)\nXtrain = scaler.transform(Xtrain)\nXtest = scaler.transform(Xtest)\n\nfrom sklearn.neural_network import MLPClassifier\nscaler = StandardScaler()\nmodel = MLPClassifier(hidden_layer_sizes=(len(features),len(features),len(features)))\n'

In [51]:
# Score model
model.score(X_test, Y_test) #TODO TODO TODO TODO TODO TODO TODO TODO

0.7968415357595394

# Predict for Testing data

In [52]:
# Clean Data
MyDataFrame = pd.read_csv('avocado-test.csv', index_col='id')
# Drops missing values 
#MyDataFrame.dropna()
# Convert strings to numbers
c = ['TotalVolume',"4046","4225","4770","TotalBags","SmallBags","LargeBags","XLargeBags"]#,"AveragePrice"]
for column in c:
    MyDataFrame[column] = pd.to_numeric(MyDataFrame[column])

TypeError: oneHotEncoding() takes from 2 to 3 positional arguments but 4 were given

In [72]:
# Select features
XforPredictions =  MyDataFrame[my_cols]#features]#None #TODO TODO TODO TODO TODO TODO TODO TODO

#my_pipeline.fit(X_train, Y_train)
# Preprocessing of validation data, get predictions


#X_train, X_test = oneHotEncoding(X, XforPredictions)
#X_train, X_test = imputePlus(X_train ,X_test)

#XforPredictions.head()
X_test.head()

,type,TotalVolume,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags
id,,,,,,,,,
10284,organic,1573.28,3.71,39.58,0.00,1529.99,1529.99,0.00,0.00
5461,conventional,29569086.56,9422275.27,13846044.56,1112405.42,5188361.31,3938274.51,1183732.08,66354.72
1010,conventional,79513.63,6624.37,63384.56,37.48,9467.22,8958.14,509.08,0.00
2189,organic,4883.55,408.44,3284.07,0.00,1191.04,572.85,618.19,0.00
2995,organic,25757.73,3749.12,11950.72,63.86,9994.03,647.47,9346.56,0.00


In [74]:
# Make Predictions!
#model.fit(X,Y)
yPredictions =  my_pipeline.predict(XforPredictions)# model.predict(X_test)# None #TODO TODO TODO TODO TODO TODO TODO TODO
print("Predictions:", yPredictions)

Predictions: [1.9344 1.4678 1.5374 ... 1.4249 1.8776 1.6919]


In [75]:
# Save to CSV File!
XforPredictions =  MyDataFrame[features]
output = pd.DataFrame({'id': XforPredictions.index, 'AveragePrice': yPredictions})
output.to_csv('avocado-submission.csv', index=False)
print(output)

id  AveragePrice
0        0        1.9344
1        1        1.4678
2        2        1.5374
3        3        1.5029
4        4        1.4265
...    ...           ...
5470  5470        1.5770
5471  5471        1.3620
5472  5472        1.4249
5473  5473        1.8776
5474  5474        1.6919

[5475 rows x 2 columns]


# Upload for grading

In [76]:
# Download CTFSG Grader Libraries
import urllib.request, os
urllib.request.urlretrieve('https://raw.githubusercontent.com/alttablabs/ctfsg-utils/master/pyctfsglib.py', './pyctfsglib.py')
print('Downloaded pyctfsglib.py:', 'pyctfsglib.py' in os.listdir())

Downloaded pyctfsglib.py: True


In [77]:
# Connect to graders
import pyctfsglib as ctfsg
import random

USER_TOKEN = "MXhtGfdjdsUfiEKTHHEuVGohZESBdMiHrFkmYqNqIFfcWOHGvcubvHJvnxpAqRMh" # You need to fill this up
GRADER_URL = random.choice([
"http://challenges.csdc20t.ctf.sg:30001/", "http://challenges.csdc20t.ctf.sg:30002/"
])
grader = ctfsg.DSGraderClient(GRADER_URL, USER_TOKEN)

DSGraderClient: Successfully Connected!
[SERVER] MOTD: CHECK your USER_TOKEN and GRADER_URL HTTP address! I'm AVOCADO_PRICE TEST_GRADER_1


In [78]:
grader.submitFile('avocado-submission.csv')

ProofOfWork Challenge =>  ('CTFSGRB155394a824a80106bc53160d69a2e984', 22)
ProofOfWork Answer Found! =>  4815595


'{"challenge":{"name":"Avocado Prices"},"id":"ck8mqfgke6u520868usyn9kvg","status":"PARTIALLY_CORRECT","multiplier":0.7539,"submittedBy":{"username":"nyjc-1"},"createdAt":"2020-04-05T07:36:27Z"}'